pip install biopython


# Global Alignment & its Applications

# Needleman_wunsch Algorithm 

In [24]:
def needleman_wunsch(seq1, seq2, match_score, mismatch_penalty, gap_penalty):
    # Initialize the score matrix
    score_matrix = [[0] * (len(seq2) + 1) for _ in range(len(seq1) + 1)]

    # Initialize the traceback matrix
    traceback_matrix = [[None] * (len(seq2) + 1) for _ in range(len(seq1) + 1)]

    # Initialize the first row and column of the score matrix
    for i in range(len(seq1) + 1):
        score_matrix[i][0] = gap_penalty * i
        traceback_matrix[i][0] = 'up'

    for j in range(len(seq2) + 1):
        score_matrix[0][j] = gap_penalty * j
        traceback_matrix[0][j] = 'left'

    # Fill in the score matrix and traceback matrix
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            match = score_matrix[i - 1][j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_penalty)
            delete = score_matrix[i - 1][j] + gap_penalty
            insert = score_matrix[i][j - 1] + gap_penalty
            score_matrix[i][j] = max(match, delete, insert)

            if score_matrix[i][j] == match:
                traceback_matrix[i][j] = 'diagonal'
            elif score_matrix[i][j] == delete:
                traceback_matrix[i][j] = 'up'
            else:
                traceback_matrix[i][j] = 'left'

    # Traceback to find the optimal alignment
    alignment1 = ''
    alignment2 = ''
    i, j = len(seq1), len(seq2)

    while i > 0 or j > 0:
        if traceback_matrix[i][j] == 'diagonal':
            alignment1 = seq1[i - 1] + alignment1
            alignment2 = seq2[j - 1] + alignment2
            i -= 1
            j -= 1
        elif traceback_matrix[i][j] == 'up':
            alignment1 = seq1[i - 1] + alignment1
            alignment2 = '-' + alignment2
            i -= 1
        else:
            alignment1 = '-' + alignment1
            alignment2 = seq2[j - 1] + alignment2
            j -= 1

    return alignment1, alignment2, score_matrix[len(seq1)][len(seq2)]


# Example usage
seq1 ="VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGIGKKVADALTNAVAHVDDMPNALSALSBLHAHKLRVDPVNFKLLSHCLIVTLAALLPAEETPAVHASLDKFLASVSTVLTSKYE" 
seq2 ="VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGIGKKVADALTNAVAHVDDMPNALSALSBLHAHKLRVDPVNFKLLSHCLIVTLAALLPAEETPGVHASLDKFLASVSTVLTSKYF" 

match_score = 1
mismatch_penalty = -1
gap_penalty = -1

alignment1, alignment2, score = needleman_wunsch(seq1, seq2, match_score, mismatch_penalty, gap_penalty)

print("Alignment 1:", alignment1)
print("Alignment 2:", alignment2)
print("Alignment Score:", score)


Alignment 1: VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGIGKKVADALTNAVAHVDDMPNALSALSBLHAHKLRVDPVNFKLLSHCLIVTLAALLPAEETPAVHASLDKFLASVSTVLTSKYE
Alignment 2: VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGIGKKVADALTNAVAHVDDMPNALSALSBLHAHKLRVDPVNFKLLSHCLIVTLAALLPAEETPGVHASLDKFLASVSTVLTSKYF
Alignment Score: 137


# Drug Target Identification

In [10]:
from Bio import pairwise2
from Bio import SeqIO

# Load the target protein sequence
target_sequence = SeqIO.read("C:\\Users\\thumm\\Downloads\\rcsb_pdb_1A3N.fasta", "fasta")

# Load the reference sequences (e.g., from a database or a file)
reference_sequences_path1 = "C:\\Users\\thumm\\Desktop\\IBS-04 project\\rcsb_pdb_1MSO.fasta"
reference_sequences_path2 = "C:\\Users\\thumm\\Desktop\\IBS-04 project\\rcsb_pdb_1MSO.fasta"
reference_sequences1 = SeqIO.parse(reference_sequences_path1, "fasta")
reference_sequences2 = SeqIO.parse(reference_sequences_path2, "fasta")

# Define a threshold for sequence similarity
similarity_threshold = 70

# Flag to check if any potential drug targets were found
found_potential_drug_targets = False

# Perform global alignment for each reference sequence in the first set
for reference_sequence in reference_sequences1:
    alignment = pairwise2.align.globalxx(target_sequence.seq, reference_sequence.seq, one_alignment_only=True)[0]
    target_aligned = alignment.seqA
    reference_aligned = alignment.seqB
    similarity = (alignment.score / len(target_aligned)) * 100

    # Check if the similarity meets the threshold
    if similarity >= similarity_threshold:
        found_potential_drug_targets = True
        print("Potential drug target found!")
        print("Target sequence ID:", target_sequence.id)
        print("Reference sequence ID:", reference_sequence.id)
        print("Similarity:", similarity)
        print("Aligned target sequence:", target_aligned)
        print("Aligned reference sequence:", reference_aligned)
        print("Alignment score:", alignment.score)
        print("---")

# Perform global alignment for each reference sequence in the second set
for reference_sequence in reference_sequences2:
    alignment = pairwise2.align.globalxx(target_sequence.seq, reference_sequence.seq, one_alignment_only=True)[0]
    target_aligned = alignment.seqA
    reference_aligned = alignment.seqB
    similarity = (alignment.score / len(target_aligned)) * 100

    # Check if the similarity meets the threshold
    if similarity >= similarity_threshold:
        found_potential_drug_targets = True
        print("Potential drug target found!")
        print("Target sequence ID:", target_sequence.id)
        print("Reference sequence ID:", reference_sequence.id)
        print("Similarity:", similarity)
        print("Aligned target sequence:", target_aligned)
        print("Aligned reference sequence:", reference_aligned)
        print("Alignment score:", alignment.score)
        print("---")

# Check if any potential drug targets were found
if not found_potential_drug_targets:
    print("No potential drug targets found.")


No potential drug targets found.


# Local Alignment & its Applications

# Smith_Waterman Algorithm

In [23]:
def smith_waterman(seq1, seq2, match_score, mismatch_penalty, gap_penalty):
    # Initialize the score matrix
    score_matrix = [[0] * (len(seq2) + 1) for _ in range(len(seq1) + 1)]

    # Initialize the traceback matrix
    traceback_matrix = [[None] * (len(seq2) + 1) for _ in range(len(seq1) + 1)]

    # Initialize variables to keep track of the maximum score and its position
    max_score = 0
    max_pos = (0, 0)

    # Fill in the score matrix and traceback matrix
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            match = score_matrix[i - 1][j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_penalty)
            delete = score_matrix[i - 1][j] + gap_penalty
            insert = score_matrix[i][j - 1] + gap_penalty
            score_matrix[i][j] = max(match, delete, insert, 0)

            if score_matrix[i][j] == match:
                traceback_matrix[i][j] = 'diagonal'
            elif score_matrix[i][j] == delete:
                traceback_matrix[i][j] = 'up'
            elif score_matrix[i][j] == insert:
                traceback_matrix[i][j] = 'left'
            else:
                traceback_matrix[i][j] = 'stop'

            # Update the maximum score and its position
            if score_matrix[i][j] > max_score:
                max_score = score_matrix[i][j]
                max_pos = (i, j)

    # Traceback to find the optimal local alignment
    alignment1 = ''
    alignment2 = ''
    i, j = max_pos

    while i > 0 and j > 0 and traceback_matrix[i][j] != 'stop':
        if traceback_matrix[i][j] == 'diagonal':
            alignment1 = seq1[i - 1] + alignment1
            alignment2 = seq2[j - 1] + alignment2
            i -= 1
            j -= 1
        elif traceback_matrix[i][j] == 'up':
            alignment1 = seq1[i - 1] + alignment1
            alignment2 = '-' + alignment2
            i -= 1
        else:
            alignment1 = '-' + alignment1
            alignment2 = seq2[j - 1] + alignment2
            j -= 1

    return alignment1, alignment2, max_score

# Example usage
seq1 = "VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKYR"
seq2 = "GIVEQCCTSICSLYQLENYCN"
match_score = 1
mismatch_penalty = -1
gap_penalty = -1

alignment1, alignment2, score = smith_waterman(seq1, seq2, match_score, mismatch_penalty, gap_penalty)

print("Alignment 1:", alignment1)
print("Alignment 2:", alignment2)
print("Alignment Score:", score)

Alignment 1: LE
Alignment 2: LE
Alignment Score: 2


# Detection Of Sequence Variants

In [34]:
from Bio import pairwise2
from Bio import SeqIO

def detect_sequence_variants(target_sequence_file, reference_sequence_file, similarity_threshold):
    # Load the target sequence
    target_sequence = SeqIO.read(target_sequence_file, "fasta")

    # Load the reference sequences
    reference_sequences = SeqIO.parse(reference_sequence_file, "fasta")

    # Perform local alignment between the target and reference sequences
    for reference_sequence in reference_sequences:
        alignments = pairwise2.align.localxx(target_sequence.seq, reference_sequence.seq)

        # Iterate over the alignments and analyze sequence variants
        for alignment in alignments:
            target_start = alignment.start
            target_end = alignment.end
            reference_start = alignment.start
            reference_end = alignment.end

            # Extract the aligned regions
            target_aligned = alignment.seqA[target_start:target_end]
            reference_aligned = alignment.seqB[reference_start:reference_end]

            # Check if the aligned regions are different
            if target_aligned != reference_aligned:
                print("Sequence variant detected!")
                print("Target sequence ID:", target_sequence.id)
                print("Reference sequence ID:", reference_sequence.id)
                print("Variant position:", target_start)
                print("Target aligned region:", target_aligned)
                print("Reference aligned region:", reference_aligned)
                print("---")
                
 # Example usage
target_sequence_file ="C:\\Users\\thumm\\Desktop\\rcsb_pdb_1A3N.fasta"
reference_sequence_file ="C:\\Users\\thumm\\Desktop\\rcsb_pdb_1MSO.fasta"
similarity_threshold = 90
"C:\\Users\\thumm\\Desktop\\rcsb_pdb_1MSO.fasta"
detect_sequence_variants(target_sequence_file, reference_sequence_file, similarity_threshold)




Sequence variant detected!
Target sequence ID: 1A3N_1|Chains
Reference sequence ID: 1MSO_1|Chains
Variant position: 14
Target aligned region: GK-VGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGKKVADAL--TNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSH-CLLVTLAAHLPAEFTPAVHASLDKFLASVSTV--LTSK--Y
Reference aligned region: G-IV------------E-----------------------Q------------CCT----------------------------------S-IC-------------------S----L------YQL---ENY
---
Sequence variant detected!
Target sequence ID: 1A3N_1|Chains
Reference sequence ID: 1MSO_1|Chains
Variant position: 14
Target aligned region: GKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGKKVADAL--TNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSH-CLLVTLAAHLPAEFTPAVHASLDKFLASVSTV--LTSK--Y
Reference aligned region: GIV------------E-----------------------Q------------CCT----------------------------------S-IC-------------------S----L------YQL---ENY
---


# Sequence Similarity Search

In [12]:
from Bio import SeqIO
from Bio import pairwise2

def sequence_similarity_search(query_sequence_file, database_sequence_file, similarity_threshold):
    # Load the query sequence
    query_sequence = SeqIO.read( "C:\\Users\\thumm\\Downloads\\Sequence Alignment\\rcsb_pdb_1A3N - Copy.fasta","fasta")

    # Load the database sequences
    database_sequences = SeqIO.parse("C:\\Users\\thumm\\Desktop\\rcsb_pdb_1A3N.fasta", "fasta")

    # Perform local alignment for each database sequence
    for database_sequence in database_sequences:
        alignments = pairwise2.align.localxx(query_sequence.seq, database_sequence.seq)

        # Iterate over the alignments and check similarity
        for alignment in alignments:
            alignment_length = alignment.end - alignment.start
            similarity = (alignment.score / alignment_length) * 100

            # Check if similarity meets the threshold
            if similarity >= similarity_threshold:
                print("Similar sequence found!")
                print("Query sequence ID:", query_sequence.id)
                print("Database sequence ID:", database_sequence.id)
                print("Similarity:", similarity)
                print("---")
               
# Example usage
query_sequence_file = "C:\\Users\\thumm\\Downloads\\Sequence Alignment\\rcsb_pdb_1A3N - Copy.fasta"
database_sequence_file ="C:\\Users\\thumm\\Desktop\\rcsb_pdb_1A3N.fasta"
similarity_threshold = 80

sequence_similarity_search(query_sequence_file, database_sequence_file, similarity_threshold)


Similar sequence found!
Query sequence ID: 1A3N_1|Chains
Database sequence ID: 1A3N_1|Chains
Similarity: 93.10344827586206
---
Similar sequence found!
Query sequence ID: 1A3N_1|Chains
Database sequence ID: 1A3N_1|Chains
Similarity: 93.75
---
Similar sequence found!
Query sequence ID: 1A3N_1|Chains
Database sequence ID: 1A3N_1|Chains
Similarity: 93.75
---
Similar sequence found!
Query sequence ID: 1A3N_1|Chains
Database sequence ID: 1A3N_1|Chains
Similarity: 94.4055944055944
---
Similar sequence found!
Query sequence ID: 1A3N_1|Chains
Database sequence ID: 1A3N_1|Chains
Similarity: 93.75
---
Similar sequence found!
Query sequence ID: 1A3N_1|Chains
Database sequence ID: 1A3N_1|Chains
Similarity: 94.4055944055944
---
Similar sequence found!
Query sequence ID: 1A3N_1|Chains
Database sequence ID: 1A3N_1|Chains
Similarity: 94.4055944055944
---
Similar sequence found!
Query sequence ID: 1A3N_1|Chains
Database sequence ID: 1A3N_1|Chains
Similarity: 95.07042253521126
---
Similar sequence found!